<a href="https://colab.research.google.com/github/enesbasbug/Data_Science_Projects/blob/main/Stock_Market_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Market Performance Analysis

Stock Market Performance Analysis involves calculating moving averages, measuring volatility, conducting correlation analysis and analyzing various aspects of the stock market to gain a deeper understanding of the factors that affect stock prices and the relationships between the stock prices of different companies.

Let’s start the task of Stock Market Performance Analysis by importing the necessary Python libraries and the dataset. For this task, I will use the Yahoo finance API (yfinance) to collect real-time stock market data for the past three months.

In [2]:
# !pip install yfinance

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

Download the historical stock price data for four companies: Apple, Microsoft, Netflix, and Google, for the last three months.

In [7]:
tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
df.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-02  144.380005  146.710007  143.899994  145.910004  145.708466   
       2023-03-03  148.039993  151.110001  147.330002  151.029999  150.821381   
       2023-03-06  153.789993  156.300003  153.460007  153.830002  153.617523   
       2023-03-07  153.699997  154.029999  151.130005  151.600006  151.390610   
       2023-03-08  152.809998  153.470001  151.830002  152.869995  152.658844   

                     Volume  
Ticker Date                  
AAPL   2023-03-02  52238100  
       2023-03-03  70732300  
       2023-03-06  87558000  
       2023-03-07  56182000  
       2023-03-08  47204800

In [10]:
df = df.reset_index()
df.head(2)

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2023-03-02,144.380005,146.710007,143.899994,145.910004,145.708466,52238100
1,AAPL,2023-03-03,148.039993,151.110001,147.330002,151.029999,150.821381,70732300


Have a look at the performance in the stock market of all the companies:

In [11]:
import plotly.express as px
fig = px.line(df, x='Date', y='Close', color='Ticker', title='Stock Market Performance for the last 3 months')
fig.show()

Compare the performance of different companies and identify similarities or differences in their stock price movements:

In [12]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

Analyse moving averages, which provide a useful way to identify trends and patterns in each company’s stock price movements over a period of time:



In [13]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
59  173.020003  171.870000
60  173.306001  172.157500
61  173.829001  172.543001
62  174.347000  172.978501
63  175.041499  173.587751

[64 rows x 2 columns]
Moving Averages for GOOG
        MA10       MA20
192      NaN        NaN
193      NaN        NaN
194      NaN        NaN
195      NaN        NaN
196      NaN        NaN
..       ...        ...
251  121.837  115.15925
252  122.588  116.01975
253  123.356  116.86625
254  123.684  117.73575
255  123.921  118.62225

[64 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
..          ...         ...
123  315.723999  311.699998
124  318.116000  312.981499
125  320.291000

Visualize the moving averages of all companies:


In [14]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  title=f"{ticker} Moving Averages")
    fig.show()

The output shows four separate graphs for each company. When the MA10 crosses above the MA20, it is considered a bullish signal indicating that the stock price will continue to rise. Conversely, when the MA10 crosses below the MA20, it is a bearish signal that the stock price will continue falling.

Analyse the volatility of all companies. Volatility is a measure of how much and how often the stock price or market fluctuates over a given period of time. Here’s how to visualize the volatility of all companies:

In [15]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

High volatility indicates that the stock or market experiences large and frequent price movements, while low volatility indicates that the market experiences smaller or less frequent price movements.

Analysing the correlation between the stock prices of Apple and Microsoft:

In [16]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()

The stock prices of Apple and Microsoft exhibit a robust linear connection, implying that when Apple's stock price rises, Microsoft's stock price also tends to increase. This indicates a substantial correlation or similarity between the two companies, likely influenced by factors like industry trends, market conditions, or shared business partners and customers. From an investor's perspective, this positive correlation presents an opportunity to diversify their portfolio by investing in both companies. Such a strategy allows for potential returns and risks that are similarly aligned, offering an attractive investment proposition.